# Tutorial : Building a Multi-Agent Chat System with Azure OpenAI, Autogen and Tracing using Phoenix

This tutorial will guide you through building a multi-agent chat system using Azure OpenAI deployments, Autogen, and Phoenix for tracing and debugging. The example scenario is a customer query resolver system, where multiple AI agents, each with a specialized role, collaboratively address user queries. You will learn how to coordinate these agents in a Round-Robin fashion, trace their interactions using Phoenix, and incorporate human annotations for debugging, optimization, and improvement.

The applications of multi-agent systems are broad, ranging from customer support and recommendation engines to dynamic problem-solving workflows. This tutorial highlights how to leverage Autogen's AgentChat framework, which provides a set of preconfigured agents with variations in behavior and response strategies. The agents in this example are capable of handling both text messages and multi-modal messages, making the system versatile and scalable.

For more details about Autogen’s AgentChat capabilities and configurations, refer to the <a href = https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html>Autogen Documentation</a>

Prerequisites

Programming Language: Python (version 3.8 or later recommended).
Azure OpenAI Access: Ensure you have access to OpenAI models via Azure OpenAI Service, including necessary credentials and deployment configurations.
Required Dependencies:
Install all necessary Python packages by running the following command:
pip install -r /path/to/requirements.txt


In [1]:
#It is a good practice to save all api keys, endpoints and other important details in an env file. 
from dotenv import load_dotenv
_ = load_dotenv("env")

Launch the Phoenix App

In [2]:
# Launch Phoenix
import os
if "PHOENIX_API_KEY" in os.environ:
    os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

else:
    import phoenix as px

    px.launch_app().view()

# Connect to Phoenix
from phoenix.otel import register
tracer_provider = register()


/workspaces/Arize_Tutorial/my_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Instrument OpenAI for tracing

In [ ]:
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

Understanding Model Clients in Autogen
In multi-agent systems, agents often rely on external LLMs to process and generate responses. Since each LLM provider (e.g., OpenAI, Azure OpenAI, local models) has distinct APIs, managing these differences can be challenging. To address this, autogen-core defines a protocol for model clients, providing a consistent interface for interacting with LLM services.

To complement this, autogen-ext includes prebuilt model clients for popular LLM providers, such as:

OpenAI GPT models: Standard GPT models hosted on OpenAI's API.
Azure OpenAI: GPT models deployed on Microsoft's Azure platform.
Local models: Custom LLMs deployed locally, allowing offline access.
These abstractions allow AgentChat to work seamlessly with different model backends, ensuring flexibility and scalability.

Setting Up Azure OpenAI with AgentChat
When using Azure OpenAI with AgentChat, you need to configure the model client to interact with your Azure Cognitive Services deployment. Here’s a breakdown of the required settings:

1. Deployment ID
The unique identifier for your deployed Azure OpenAI model.
Example: gpt-4-deployment.
2. Azure Cognitive Services Endpoint
The base URL for your Azure Cognitive Services resource.
Example: https://<your-resource-name>.openai.azure.com/.
3. API Version
Specifies the API version to use. Ensure compatibility with your Azure deployment.
Example: 2023-05-15.
4. Model Capabilities
Describes the specific functions of the model, such as text generation, summarization, or classification.
5. Authentication
Two methods are supported for authentication:
API Key: Use the AZURE_OPENAI_API_KEY environment variable to provide the key.
Azure Active Directory (AAD): Use an AAD token credential for enhanced security and role-based access control.

## Step 1 : Set up your Azure OpenAI Client

In [ ]:
import os
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

 

#Set up Azure OpenAI Configuration using environment variables

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

AZURE_OPENAI_API_VERSION = os.getenv("API_VERSION")

 
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="model_name",
    model="model_name",
    api_version="your_model_version",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    temperature=0.2,
    max_tokens=200
)


## Step 2 : Define Agent Functions

In this step, we define the functions for each agent in our multi-agent system. Each agent is initialized using the AssistantAgent class from Autogen. The agents' behavior is primarily determined by their system messages, which define the roles, responsibilities, and tone of each agent.

For this tutorial, we'll focus on three distinct agents:

Classifier Agent: Identifies the type of user query.
Resolver Agent: Provides a solution to the query.
Feedback Agent: Evaluates the resolution and suggests improvements.


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination # type: ignore
from autogen_agentchat.teams import RoundRobinGroupChat 

classifier_agent = AssistantAgent(

    name="classifier",
    model_client =az_model_client,

    system_message=("You are a classifier that determines the category og a customer query"
                    "The categories are Billing, Technical Support, Shipping or General Inquiry"
                    "Respond only with categoy name")
)


problem_solver_agent = AssistantAgent(

    name="problem_solver",

    model_client = az_model_client,

    system_message=("You are a resolver that answers customer queries based on their category."
                    "You will receive the category and query, and you must provide a resolution"
                    "Be concise, clear, and empatheic in your response"
                    "Try to limit the answers in 5 lines")

)

feedbackagent = AssistantAgent(
    name = "feedback_agent",
     model_client=az_model_client,
    system_message=(
        "You analyse customer interactions for sentiments and suggest improvements"
        "If the sentiment is negative, identify and propose better response"
        "If all looks good respond with TERMINATE"
    )
)

 

## Step 3 : Setting Up a Team with RoundRobinGroupChat

In this step, we configure a RoundRobinGroupChat to enable multiple agents to work collaboratively. This configuration ensures that each agent takes turns responding to user queries while maintaining a shared conversation context. To manage the flow of the conversation effectively, we'll also implement a TextMentionTermination condition, which stops the interaction when a specific keyword or phrase is detected in any agent's response.



In [5]:
termination =TextMentionTermination("TERMINATE")
group_chat1 = RoundRobinGroupChat(
    [classifier_agent, problem_solver_agent, feedbackagent], termination_condition=termination)

In [7]:
from phoenix.trace import TraceDataset
from autogen_agentchat.ui import Console


In [ ]:
query = "who do i contact for broken screen?"
await Console(group_chat1.run_stream(task=query))


In [ ]:
query1 = "My payment isnt going through on your app"
await Console(group_chat2.run_stream(task=f"Query: {query1}"))

In [ ]:
query2 = "I cant log into my account"
await Console(group_chat2.run_stream(task=f"Query: {query2}"))

## Step 4 : Open the phoenix url that opened in your localhost, you will now see the traces for all the queries that we ran


<img src = "img7.png">

## Step 5 : Adding Human Annotations

In this step, we'll explore how to add human annotations to traces in Phoenix. Human annotations serve as an essential tool for improving LLM-driven applications by capturing qualitative insights, curating datasets, and ensuring the application’s continuous improvement.

Annotations in Phoenix are simple yet effective. They allow you to assign labels or scores to specific spans (segments of a trace). These annotations can be used for debugging, sharing feedback within a team, and creating datasets for fine-tuning or training LLMs.

Purpose of Human Annotations
Quality Assurance: Provide a secondary layer of validation by tagging responses that meet or fail to meet quality expectations.
Dataset Curation: Identify good or bad examples to build curated datasets for further training.
Collaborative Debugging: Share annotations across teams to foster collaborative debugging and optimization.
LLM Judge Training: Use annotated spans to train LLMs to evaluate future interactions more effectively.


Simply click on the Trace and then click on 'Annotate'. Next give a suitable name to your annotation, provide label and a score for your reference. Refer the images below or visit this <a href=https://docs.arize.com/phoenix/tracing/llm-traces/how-to-annotate-traces>link</a> for a quickstart

<img src = "img2.png">

<img src ="img1.png">

Once done they look like this

<img src = "img3.png">


You can add more than one annotation for each trace

<img src="img6.png">

You can also Filter your traces based on the annotations, or any other filter conditions to easliy view your traces. Click on the '+' in the search bar and add the filter

<img src = "img5.png">
<img src = "img4.png">

## Step 6 : Save and Load Traces for Reuse

Saving and loading traces is a critical aspect of maintaining a robust and iterative development process when working with multi-agent systems. Phoenix provides functionality to persist traces, allowing you to analyze them later, share them across teams, or use them as benchmarks in future evaluations.

Why Save and Load Traces?
Reusability: Analyze historical interactions to identify trends or recurring issues.
Collaboration: Share traces with team members for debugging or quality assurance.
Benchmarking: Use traces to compare the performance of newer iterations or models.
Scalability: Persist and manage traces efficiently for large-scale systems.


In [ ]:
my_traces = px.Client().get_trace_dataset().save()

In [17]:
px.launch_app(trace=px.TraceDataset.load(my_traces))

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
